In [ ]:
#Importação de bibliotecas

In [ ]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import ExtraTreesRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics
import pickle
import os

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv("/kaggle/input/car-price-estimation/datasets_33080_43333_car data.csv")

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.sample(6)

In [ ]:
final_dataset = df[['Year', 'Selling_Price', 'Present_Price', 'Kms_Driven', 'Fuel_Type', 'Seller_Type', 'Transmission', 'Owner']]

In [ ]:
final_dataset.head()

In [ ]:
final_dataset['Current_Year'] = 2020

In [ ]:
final_dataset['Num_of_Years'] = final_dataset['Current_Year'] - final_dataset['Year']

In [ ]:
final_dataset.head()

In [ ]:
final_dataset.drop(['Num_of_Years'],axis=1,inplace=True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset = pd.get_dummies(final_dataset, drop_first = True)

In [ ]:
final_dataset.head()

In [ ]:
final_dataset = final_dataset.drop(['Current_Year'], axis=1)

In [ ]:
sns.pairplot(final_dataset)

In [ ]:
#Correlação entre as variáveis
final_dataset.corr()

In [ ]:
#Correlações entre as features do dataset
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#Plotagem do  heat map
g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

In [ ]:
X = final_dataset.iloc[:,1:]
y = final_dataset.iloc[:,0]

In [ ]:
X['Owner'].unique()

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
#Extração de feature importance
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
#Plotagem das features mais relevantes
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(6).plot(kind='barh')
plt.show()

In [ ]:
#Split de treino e teste
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
regressor = RandomForestRegressor()

In [ ]:
import numpy as np

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
print(n_estimators)

In [ ]:
#Randomized Search CV

n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)] #total de árvores na random forest
max_features = ['auto', 'sqrt'] #número de  features a se considerar em cada split
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)] #máximo de níveis por árvore
# max_depth.append(None)
min_samples_split = [2, 5, 10, 15, 100] 
min_samples_leaf = [1, 2, 5, 10] 


In [ ]:
# Criação do random grid, para seleção de hiperparâmetros
random_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
rf = RandomForestRegressor()

In [ ]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
print(n_estimators)

In [ ]:
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(X_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
predictions = rf_random.predict(X_test)

In [ ]:
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
#Métricas:
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))